In [1]:
import numpy as np
import pandas as pd
import itertools


from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

### 전처리가 이미 완료된 후의 데이터를 사용
보안상 데이터를 전체 공개 불가

In [2]:
df = pd.read_csv('../data/dataset/CAI_data_003.csv')

In [3]:
df['sentence'][1]

'고객 요청사항 수신불인터넷 연결오류 뜸재부팅해 봄 모텔 영업용 일중 꼭 점검원하여 지사 이관함회선 단말특이사항 진단 결과 기타 추가 연락처'

In [4]:
okt = Okt()

tokenized_doc = okt.pos(df['sentence'][1])
tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] != 0])

print('품사 태깅 10개만 출력 :',tokenized_doc[:10])
print('추출 :',tokenized_nouns)

품사 태깅 10개만 출력 : [('고객', 'Noun'), ('요청', 'Noun'), ('사항', 'Noun'), ('수', 'Modifier'), ('신', 'Modifier'), ('불', 'Noun'), ('인터넷', 'Noun'), ('연결', 'Noun'), ('오류', 'Noun'), ('뜸', 'Noun')]
추출 : 고객 요청 사항 수 신 불 인터넷 연결 오류 뜸 재부팅 해 봄 모텔 영업 용 일 중 꼭 점검 원하여 지사 이 관함 회선 단말 특이 사항 진단 결과 기타 추가 연락처


In [5]:
n_gram_range = (2, 3)

count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
candidates = count.get_feature_names_out()

print('trigram 개수 :',len(candidates))
print('trigram 다섯개만 출력 :',candidates[:5])

trigram 개수 : 41
trigram 다섯개만 출력 : ['결과 기타' '결과 기타 추가' '고객 요청' '고객 요청 사항' '관함 회선']


In [6]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
doc_embedding = model.encode([df['sentence'][1]])
candidate_embeddings = model.encode(candidates)

In [7]:
top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
print(keywords)

['점검 원하여', '연결 오류 재부팅', '원하여 지사 관함', '점검 원하여 지사', '영업 점검 원하여']


In [8]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings,
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None

    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [9]:
sentences = []

stopwords = ['고객','요청사','항','요청사항','안심권유','기타 추가 연락처']

for sentence in df['sentence']:
    for word in stopwords:
        sentence = sentence.replace(word, '')
        sentence = sentence.lstrip()
    sentences.append(sentence)
df['sen'] = sentences

In [10]:
df['sen']

0                                            안됨상담 중 자연회복됨
1       수신불인터넷 연결오류 뜸재부팅해 봄 모텔 영업용 일중 꼭 점검원하여 지사 이관함회선...
2                                  수신불회선 단말특이사 진단 결과  연후방
3                                       수신 불네트워크 오류 접수 보류
4                              수신불 연락 후회선 단말특이사 진단 결과 인 입
                              ...                        
8661    화질 불량 시청 중 화면 멈추고 꺼졌다 켜지는 현상이 있다고 함 연락 후 방문요회선...
8662                       리콜넷플릭스 연령인증 문의 건 리콜하니 해결했다고 하심
8663                                           대수신불전송 후리셋
8664    빠른 점검 요청   시청 불인터넷 연결오류나 옴 모든 장비 재부팅 안내회선 단말특이...
8665      수신불 연락 후 방문지니 셋 탑 전원안 들어옴 교체 요청회 선 단말특이사 진단 결과 
Name: sen, Length: 8666, dtype: object

In [11]:
abstract_sen = []


for i in range(len(df)) :
    

    try :
        okt = Okt()

        tokenized_doc = okt.pos(df['sen'][i])
        tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] != 0])

        n_gram_range = (2, 3)

        count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
        candidates = count.get_feature_names_out()

        doc_embedding = model.encode([df['sen'][i]])
        candidate_embeddings = model.encode(candidates)

        distances = cosine_similarity(doc_embedding, candidate_embeddings)
        keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
        
        
        keywords = max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=3, nr_candidates=10)
        
        
        abstract_sen.append(keywords)
        print('----------------------------------------------------------')
        print('원본 : ', df['sen'][i])
        print('요약 : ', abstract_sen[i])
        print('----------------------------------------------------------')

    except :
        abstract_sen.append(df['sen'][i])

----------------------------------------------------------
원본 :  안됨상담 중 자연회복됨
요약 :  ['자연 회복', '상담 자연', '안됨 상담']
----------------------------------------------------------
----------------------------------------------------------
원본 :  수신불인터넷 연결오류 뜸재부팅해 봄 모텔 영업용 일중 꼭 점검원하여 지사 이관함회선 단말특이사 진단 결과 
요약 :  ['오류 재부팅 모텔', '점검 원하여', '모텔 영업 점검']
----------------------------------------------------------
----------------------------------------------------------
원본 :  수신불회선 단말특이사 진단 결과  연후방
요약 :  ['특이 진단 결과', '결과 후방', '회선 단말']
----------------------------------------------------------
----------------------------------------------------------
원본 :  수신 불네트워크 오류 접수 보류
요약 :  ['네트워크 오류', '수신 네트워크', '접수 보류']
----------------------------------------------------------
----------------------------------------------------------
원본 :  수신불 연락 후회선 단말특이사 진단 결과 인 입
요약 :  ['특이 진단 결과', '회선 단말', '단말 특이']
----------------------------------------------------------
---------------------------------------------------

----------------------------------------------------------
원본 :  수신 불방쪽만 안됨 셋탑재 부팅 후 정상
요약 :  ['안됨 탑재', '수신 불방쪽', '탑재 부팅 정상']
----------------------------------------------------------
----------------------------------------------------------
원본 :  수신불인터넷 연결오류 뜸재부팅해 봄 연락 먼저 회선 단말특이사 진단 결과 
요약 :  ['진단 결과', '연결 오류 재부팅', '먼저 회선']
----------------------------------------------------------
----------------------------------------------------------
원본 :  가능한 일방 요 분 정도 시청되면 화면 꺼짐 소리 정상회선 단말특이사 진단 결과  코로나 미 대상
요약 :  ['진단 결과 코로나', '정도 시청 되면', '화면 꺼짐 소리']
----------------------------------------------------------
----------------------------------------------------------
원본 :  일반 아들 방안됨 전원 적색점 등재부팅해도 동일 기량 자 방문 요청함 방문 시간 참고
요약 :  ['일반 아들 방안', '동일 기량 방문', '기량 방문 요청']
----------------------------------------------------------
----------------------------------------------------------
원본 :  안됨 셋 탑 전원 빨간불껏 다 켜도 아무 반응 없음 최대한 빠른 방문 요청
요약 :  ['최대한 빠른 방문', '없음 최대한', '켜도 아무 반응']
------------------------

----------------------------------------------------------
원본 :  수신불 연락 후 전원회선 단말특이사 진단 결과 인 입
요약 :  ['특이 진단', '전원 회선 단말', '연락 전원']
----------------------------------------------------------
----------------------------------------------------------
원본 :  리모컨 작동 불리모컨에 버튼을 안 눌러도 리모컨에 불이 계속 들어오면서 오작동이 됨연 후방 회선 단말특이사 진단 결과 
요약 :  ['리모컨 계속 들어오면서', '작동 후방 회선', '리모컨 버튼']
----------------------------------------------------------
----------------------------------------------------------
원본 :  수신불인터넷 오류회선 단말특이사 진단 결과  연후방
요약 :  ['진단 결과', '오류 회선', '회선 단말']
----------------------------------------------------------
----------------------------------------------------------
원본 :  수신 불네트워크 오류 홈 너무 뜨겁다며 문제 있는 건 아닌 지 방문해서 확인 요회선 단말특이사 진단 결과 
요약 :  ['있는 아닌 방문', '해서 확인', '오류 너무 뜨겁다며']
----------------------------------------------------------
----------------------------------------------------------
원본 :  리모컨 안됨 가까이에서는 되고 멀리서는 안됨 리모컨 페어링재부팅에 택배 접수함
요약 :  ['서는 안됨 리모컨', '재부팅 택배 접수', '택배 접수']
---------

----------------------------------------------------------
원본 :  자연회복회선 단말특이사 진단 결과 
요약 :  ['특이 진단', '회선 단말', '자연 회복']
----------------------------------------------------------
----------------------------------------------------------
원본 :  장비 장애 안내
요약 :  ['장애 안내', '장비 장애', '장비 장애 안내']
----------------------------------------------------------
----------------------------------------------------------
원본 :  화질이 상장비리셋 안내 후 정상회선 단말특이사 진단 결과 정상기 타 추가 연락처
요약 :  ['안내 정상 회선', '추가 연락처', '화질 장비 리셋']
----------------------------------------------------------
----------------------------------------------------------
원본 :  셋탑박스 전원선을 뺐다 꼽아야 전원이 켜진다고 함 연락 후 방문회선 단말 특이사 진단 결과 
요약 :  ['꼽아 전원', '셋탑박스 원선 뺐다', '꼽아 전원 켜진다고']
----------------------------------------------------------
----------------------------------------------------------
원본 :  티비끊김회선 단말특이사 진단 결과 
요약 :  ['티비 끊김 회선', '특이 진단 결과', '회선 단말']
----------------------------------------------------------
--------------------------------------

----------------------------------------------------------
원본 :  리셋 시청 불수신 미약회선 단말특이사 진단 결과 
요약 :  ['회선 단말', '미약 회선 단말', '시청 불수신 미약']
----------------------------------------------------------
----------------------------------------------------------
원본 :  수신 불셋탑리셋동일회선단말특이사 진단 결과 
요약 :  ['단말 특이 진단', '수신 불셋탑', '리셋 동일 회선']
----------------------------------------------------------
----------------------------------------------------------
원본 :  리모컨지참까지 해서 방문 요청함 추가일 오전 시시 방문 안내함수 신불 화면 안나 옴셋탑재 부팅해도 동일 증상 언급
요약 :  ['방문 안내 함수', '부팅 해도 동일', '참까지 해서']
----------------------------------------------------------
----------------------------------------------------------
원본 :  아트빌 수신불시 방문 요청 외부 회선 단말특이사 임의 코로나 이상무진단 결과 
요약 :  ['코로나 이상무', '수신 불시 방문', '요청 외부']
----------------------------------------------------------
----------------------------------------------------------
원본 :  리모컨 잘못 건드리고 수신 불회선 단말특이사 진단 결과  엔지니어 확인되면 연락 먼저 요청
요약 :  ['건드리고 수신 회선', '리모컨 잘못 건드리고', '되면 연락 먼저']
-----------------

...

----------------------------------------------------------
원본 :  연락 먼저 필   시청 불인터넷 연결오류나 옴 모든 장비 재부팅 안내회선 단말특이사 진단 결과  통화 안 되면 문자 요청
요약 :  ['모든 장비', '결과 통화 되면', '통화 되면 문자']
----------------------------------------------------------
----------------------------------------------------------
원본 :  수신불로 딩 중리셋
요약 :  ['수신 불로 리셋', '수신 불로', '불로 리셋']
----------------------------------------------------------
----------------------------------------------------------
원본 :  빠른 방문요   수신 불네트워크 연결오류 포트리셋 전체 리셋코로나 이상 무재확인필 회선 단말특이사 진단 결과 모뎀 정상기 타 추가 연락처인
요약 :  ['결과 모뎀 정상기', '리셋 전체 리셋', '추가 연락처']
----------------------------------------------------------
----------------------------------------------------------
원본 :  리모컨 안됨 매장 방문 거부 택배 접수회선 단말특이사 진단 결과 
요약 :  ['회선 단말 특이', '거부 택배 접수', '매장 방문']
----------------------------------------------------------
----------------------------------------------------------
원본 :  수신불접수 안내했으나 보류하심회선 단말특이사 진단 결과 코로나 이상무
요약 :  ['접수 했으나', '단말 특이 진단', '코로나 이상무']
-------

----------------------------------------------------------
원본 :  수신 불재전송 후 정상회선 단말특이사 진단 결과 
요약 :  ['수신 불재', '회선 단말 특이', '전송 정상']
----------------------------------------------------------
----------------------------------------------------------
원본 :  연   수신불로딩 중 만나 옴장 비리셋 안내 코로나 이 상무회선 단말특이사 진단 결과 
요약 :  ['회선 단말 특이', '안내 코로나 상무', '로딩 만나']
----------------------------------------------------------
----------------------------------------------------------
원본 :  수신 불리셋 회선 단말특이사 진단 결과 
요약 :  ['특이 진단', '수신 리셋 회선', '단말 특이']
----------------------------------------------------------
----------------------------------------------------------
원본 :  나오다 말다시 청 중끊김 저녁만 되면 자주 안 돼 불편회선 단말특이사 코로나무 진단 결과 
요약 :  ['다시 끊김 저녁', '자주 불편 회선', '저녁 되면 자주']
----------------------------------------------------------
----------------------------------------------------------
원본 :  안됨 셋탑재 부팅해도 안됨화 요일시 반시까지 시간 가능 안 될 시 다음 주 화요일까지 기다려야 하며 평일엔 저녁시 이후 시간 가능 일점검 안 될 시 해지 요청함 익일 의뢰 조정 일정 문자전송 안내
요약 :  ['평일 저녁 이후', '

----------------------------------------------------------
원본 :  일 오후 시안내함 티비수 신불회선 중 에어셋탑 회선화질이상재부팅 안내함
요약 :  ['에어 셋탑', '회선 화질 이상', '오후 시안 티비']
----------------------------------------------------------
----------------------------------------------------------
원본 :  수신 불통화 중 끊김리콜 통화 중 문자 발송인 입회선단말특이사
요약 :  ['끊김 리콜', '리콜 통화 문자', '발송 입회 선단']
----------------------------------------------------------
----------------------------------------------------------
원본 :  리모컨 모든 버튼 작동 불건전지 교체 확인 램프적색점 등 셋탑재 부팅 후 동일  거주 지역 매장 없음택배 배송회선 단말특이사 진단 결과 올인 홈모뎀 정상기 타 추가 연락처
요약 :  ['모뎀 정상기', '리모컨 모든 버튼', '탑재 부팅 동일']
----------------------------------------------------------
----------------------------------------------------------
원본 :  기가 지니티 비출력 쪽으로 음성안 나 옴기가 지니스피커로는 정상됨 회선 단말특이사 진단 결과 
요약 :  ['회선 단말 특이', '로는 정상', '스피커 로는 정상']
----------------------------------------------------------
----------------------------------------------------------
원본 :  방쪽 티비수 신불인터넷 연결오류 남선 연결 확인 셋탑리붓 동일회선단말특이사 진단 결과 
요약 

----------------------------------------------------------
원본 :  용수평수 신불로딩 중계속 공실방비번 부재시 연락 안 되도 들어와서 수리하고 가시면 된다고 함독 촉일 방문 요청 독촉 예정회선 단말특이사 진단 결과 
요약 :  ['함독 촉일 방문', '부재시 연락 되도', '중계 실방 비번']
----------------------------------------------------------
----------------------------------------------------------
원본 :  수신불회선 단말특이사 진단 결과 
요약 :  ['특이 진단 결과', '회선 단말', '단말 특이']
----------------------------------------------------------
----------------------------------------------------------
원본 :  화면 깜박거림회선 단말특이사 진단 결과 임의회 선 
요약 :  ['단말 특이 진단', '결과 의회', '깜박 거림']
----------------------------------------------------------
----------------------------------------------------------
원본 :  시청 불신 미약 최대한 오후 시쯤 방문 요망 어려우면 미리 연락주면 방키 알려주신 다함 회선 단말특이사 진단 결과 
요약 :  ['미리 연락 주면', '불신 미약 최대한', '어려우면 미리 연락']
----------------------------------------------------------
----------------------------------------------------------
원본 :  위성방송안 나옴위성신 이상뜸세톱박스리셋회선 단말특이사 진단 결과 
요약 :  ['박스 리셋 회선', '이상 뜸세', '이상 뜸세 박스']
--

----------------------------------------------------------
원본 :  두 대 모두 안 나 옴 포트리셋 회선 단말특이사 진단 결과 
요약 :  ['리셋 회선', '단말 특이 진단', '모두 포트']
----------------------------------------------------------
----------------------------------------------------------
원본 :  수신불회선 단말특이사인 진단 결과 
요약 :  ['회선 단말', '특이 사인 진단', '사인 진단 결과']
----------------------------------------------------------
----------------------------------------------------------
원본 :  대화면 끊김 빠른 점검요회선단말특이사 진단 결과 
요약 :  ['단말 특이 진단', '점검 회선', '끊김 빠른']
----------------------------------------------------------
----------------------------------------------------------
원본 :  안됨상담 중 정상
요약 :  ['상담 정상', '안됨 상담 정상', '안됨 상담']
----------------------------------------------------------
----------------------------------------------------------
원본 :  대 모두 안됨 인터넷 연결오류시분 이후 방문 요청회 선 단말특이사 진단 결과 
요약 :  ['오류 이후 방문', '단말 특이', '방문 요청']
----------------------------------------------------------
--------------------------------------------------------

In [12]:
df['sentence']

0                                   고객 요청사 항 안됨상담 중 자연회복됨
1       고객 요청사항 수신불인터넷 연결오류 뜸재부팅해 봄 모텔 영업용 일중 꼭 점검원하여 ...
2                고객 요청사항 수신불회선 단말특이사항 진단 결과 기타 추가 연락처 연후방
3                                       수신 불네트워크 오류 접수 보류
4            고객 요청사항 수신불 연락 후회선 단말특이사항 진단 결과 기타 추가 연락처인 입
                              ...                        
8661    안심권유 고객 요청사항 화질 불량 시청 중 화면 멈추고 꺼졌다 켜지는 현상이 있다고...
8662                       리콜넷플릭스 연령인증 문의 건 리콜하니 해결했다고 하심
8663                                   고객 요청사항 대수신불전송 후리셋
8664    안심권유 빠른 점검 요청 고객 요청사항 시청 불인터넷 연결오류나 옴 모든 장비 재부...
8665    고객 요청사항 수신불 연락 후 방문지니 셋 탑 전원안 들어옴 교체 요청회 선 단말특...
Name: sentence, Length: 8666, dtype: object

In [13]:
abstract_sen

[['자연 회복', '상담 자연', '안됨 상담'],
 ['오류 재부팅 모텔', '점검 원하여', '모텔 영업 점검'],
 ['특이 진단 결과', '결과 후방', '회선 단말'],
 ['네트워크 오류', '수신 네트워크', '접수 보류'],
 ['특이 진단 결과', '회선 단말', '단말 특이'],
 ['포트 리셋', '미리 연락', '정상 넷플릭스 안됨'],
 ['단말 특이 진단', '수신 서비스', '시작 회선'],
 ['김현철 채팅', '자체 에서 잡음', '받았으나 소리 난다고'],
 ['진단 결과', '난다 리셋', '따닥거리 소리'],
 ['반복 연락 회선', '불량 시청', '화질 불량 시청'],
 ['특이 진단 결과', '결과 후방', '회선 단말'],
 ['방문 요망 회선', '진단 결과', '빠른 방문'],
 ['포트 리셋', '빠른 방문', '리셋 연락 최대한'],
 ['스트림 에러 회복', '진단 결과', '회선 단말'],
 ['리셋 정상', '안됨 리셋', '안됨 리셋 정상'],
 ['입수 재부팅', '정상 사용', '인터넷 지킴 입수'],
 ['실행 불가', '리셋 회선', '진단 결과'],
 ['회선 단말 특이', '진단 결과', '최대한 빠른'],
 ['안됨 리모컨', '리모컨 계속 매장', '매장 안내'],
 ['수신 불셋', '정상 확인', '수신 불셋 탑재'],
 ['정상 사용', '작동 불셋 탑재', '리모컨 작동 불셋'],
 ['들어옴 의뢰', '원안 들어옴', '원안 들어옴 의뢰'],
 ['들어옴 의뢰', '원안 들어옴', '원안 들어옴 의뢰'],
 ['단말 특이 시험', '회선 단말', '코로나 대상'],
 ['탑재 부팅', '해도 동일 보류', '안됨신 없음 외부'],
 ['진단 결과', '요청 단말 특이', '되도록 오전'],
 ['끊어짐 코로나', '회선 단말 특이', '수신 평소 화면'],
 ['연결 오류 재부팅', '동일 방문', '연결 오류'],
 ['회선 단말', '진단 결과', '정상 사용'],
 ['외부

## Max Sum Similarity
데이터 쌍 사이의 최대 합 거리는 데이터 쌍 간의 거리가 최대화되는 데이터 쌍으로 정의됩니다. 여기서의 의도는 후보 간의 유사성을 최소화하면서 문서와의 후보 유사성을 극대화하고자 하는 것입니다.